# Initiation to Chemoinformatics in Python (French).
## Analysis of the Mutag dataset (https://pubs.acs.org/doi/abs/10.1021/jm00106a046).

The goal of this intiation to chemoinformatics is the analysis of the Mutag molecule dataset. this dataset contains 188 molecules labeled 1 or −1 following presence or abscence of mutagenicity power.
(This initiation is written in French... so translation is up to you).

### 1. Lecture des données
Créer une fonction ```read_data``` qui lise un fichier de fingerprints (par exemple, ```mutag_188_CIRC_2.fingerprints```) et un fichier d'étiquettes (par exemple, ```mutag_188_target.txt```) et crée une liste ```samples``` de dictionnaires représentants les fingerprints, et une liste ```labels``` d'étiquettes.

Chaque dictionnaire correspond à une molécule du fichier de fingerprints.
Les clés de ce dictionnaire sont les indices des sous-graphes (définis par les lignes commençant par ```#```).
Pour chaque clé, la valeur correspondante est le nombre d'occurences du sous-graphe correspondant dans la molécule.


In [1]:
def read_data(in_fingerprints_f, in_labels_f):
    """
    Parameters:
      in_fingerprints_f: file path
        Fingerprints file, in the same format as mutag_188_CIRC_2.fingerprints.
      in_labels_f: file path
        Labels file, in the same format as mutag_188_target.txt
        
    Returns:
    samples: list
      List containing as many elements as there are samples (i.e. molecules) in in_fingerprints_f.
      Each element is a dictionary in which the keys are subgraph identifiers (indices in the original fingerprint file)
      and the values the number of occurrences of the corresponding subgraph in the molecule.
    labels: list
      List containing as many elements as there are samples (i.e. molecules). 
      Each element is the label of the corresponding molecule.
      Each element is either +1 or -1.
    """
    
    
    # Read samples
    samples = []
    with open(in_fingerprints_f,"r") as fingerprint:
        for line in fingerprint:
            if line[0] != '#':
                dicomol={}
                liste=line.split('\t')[2].split()
                for i in liste:
                    dicomol[i.split(':')[0]] = int(i.split(':')[1])
                samples.append(dicomol)
                    
    # Read labels
    labels = []  
    with open(in_labels_f,"r") as flab:
        for line in flab:
            labels.append(line.split()[0])    
    return samples, labels

read_data('/home/yoann/mutag_188_CIRC_d2.fingerprints','/home/yoann/mutag_188_target.txt')

([{'0': 1,
   '1': 14,
   '10': 1,
   '11': 1,
   '12': 1,
   '13': 1,
   '14': 2,
   '15': 1,
   '16': 5,
   '17': 9,
   '18': 1,
   '2': 1,
   '3': 2,
   '4': 1,
   '5': 1,
   '6': 1,
   '7': 2,
   '8': 4,
   '9': 2},
  {'0': 1,
   '1': 9,
   '10': 2,
   '11': 1,
   '13': 1,
   '14': 1,
   '16': 2,
   '17': 5,
   '18': 1,
   '19': 1,
   '2': 1,
   '20': 1,
   '21': 1,
   '22': 1,
   '23': 1,
   '24': 1,
   '25': 1,
   '26': 1,
   '27': 1,
   '4': 1,
   '5': 1,
   '6': 1,
   '8': 1,
   '9': 2},
  {'0': 1,
   '1': 9,
   '10': 2,
   '11': 1,
   '13': 1,
   '14': 1,
   '16': 1,
   '17': 5,
   '18': 1,
   '19': 1,
   '2': 1,
   '20': 1,
   '21': 1,
   '22': 1,
   '24': 1,
   '25': 1,
   '28': 1,
   '29': 1,
   '30': 1,
   '31': 1,
   '4': 1,
   '5': 1,
   '6': 1,
   '8': 1,
   '9': 2},
  {'0': 1,
   '1': 16,
   '10': 1,
   '11': 1,
   '13': 1,
   '14': 3,
   '16': 2,
   '17': 9,
   '18': 1,
   '2': 1,
   '3': 1,
   '32': 1,
   '33': 1,
   '34': 3,
   '35': 2,
   '36': 1,
   '37': 4,
   '3

### 2. Calcul de similarité (Tanimoto)
Créer une fonction ```tanimoto``` qui retourne la similarité de Tanimoto entre deux dictionnaires.

In set notation,  
$$ \mbox{Tanimoto}(A, B) = \frac{|A \cap B|}{|A \cup B|} $$


In [2]:
def tanimoto(dict_A,dict_B):
    """
    Compute the Tanimoto similarity between dict_A and dict_B.
    
    Parameters:
      dict_A: dictionary 
        the keys are subgraph identifiers (indices in the original fingerprint file)
        and values are number of occurrences of the corresponding subgraph in object A.
      dict_B: dictionary
        the keys are subgraph identifiers (indices in the original fingerprint file)
        and values are number of occurrences of the corresponding subgraph in object B.
    
    Returns:
      tan: Tanimoto coefficient of similarity between dict_A and dict_B.
    """
    tan = 0.
    
    intersect=set(dict_A.keys()).intersection(dict_B.keys())
    uni=set(dict_A.keys()).union(dict_B.keys())
    tan= float(len(intersect))/float(len(uni))
    ''' 
    try:
        assert tan >= 0. and tan <= 1.
    except AssertionError:
        print "ERROR: Tanimoto coefficient %.2e is not between 0 and 1." % tan
    '''
    
    return tan

Vérifier que la similarité de Tanimoto entre les deux premières molécules de `mutag_188_CIRC_d2.fingerprints` est de 0.535714.

In [3]:
samples, labels = read_data("/home/yoann/mutag_188_CIRC_d2.fingerprints", "/home/yoann/mutag_188_target.txt")
print tanimoto(samples[0], samples[1])

0.535714285714


### 3. Calcul de similarité (Minmax)
Créer une fonction ```minmax``` qui retourne la similarité Minmax entre deux dictionnaires.
$$\mbox{Minmax}(A, B) = \frac{\sum_{i=1}^N\min(A_i, B_i)}{\sum_{i=1}^N\max(A_i, B_i)}$$

In [4]:
def minmax(dict_A, dict_B):
    """
    Compute the MinMax similarity between dict_A and dict_B.
    
    Parameters:
      dict_A: dictionary 
        the keys are subgraph identifiers (indices in the original fingerprint file)
        and values are number of occurrences of the corresponding subgraph in object A.
      dict_B: dictionary 
        the keys are subgraph identifiers (indices in the original fingerprint file)
        and values are number of occurrences of the corresponding subgraph in object B.
    
    Returns:
      mma: Minmax coefficient of similarity between dict_A and dict_B.
    """
    
    
    mma = 0
    
    #Detailled way to set numerator and denominator
    ''' 
    intersect=set(dict_A.keys()).intersection(dict_B.keys())

    listinter=[]
    num=0
    for i in intersect:
        listinter.append(i)
    for i in listinter:
        if i in dict_A.keys():
            if i in dict_B.keys():
                if dico1[i]<=dict_B[i]:
                    num+=dict_A[i]
                elif dico1[i]>=dict_B[i]:
                    num+=dict_B[i]


    uni=set(dict_A.keys()).union(dict_B.keys())

    listuni=[]
    denom=0
    for i in uni:
        listuni.append(i)
    for i in listuni:
        if i in dict_A.keys():
            if i in dict_B.keys():
                if dict_A[i]>=dict_B[i]:
                    denom+=dict_A[i]
                elif dict_A[i]<=dict_B[i]:
                    denom+=dict_B[i]
            else:
                denom+=dict_A[i]
        else:
            denom+=dict_B[i]
    
    mma = float(num) / float(denom)
    '''    
       
    #Simpliest way to set numerator and denominator
    A_keys = set(dict_A)
    B_keys = set(dict_B)
    all_keys = A_keys.union(B_keys)

    num = 0
    denom = 0

    for key in all_keys :
        num+= min(dict_A.get(key,0), dict_B.get(key,0))
        denom+= max(dict_A.get(key,0), dict_B.get(key,0))
    mma = float(num) / float(denom)

    try:
        assert mma >= 0. and mma <= 1.
    except AssertionError:
        print "ERROR: Minmax coefficient %.2e is not between 0 and 1." % tan
    
    return mma

Vérifier que la similarité de Tanimoto entre les deux premières molécules de `mutag_188_CIRC_d2.fingerprints` est de 0.4754098.

In [5]:
#my_samples, my_labels = read_data("mutag_188_CIRC_d2.fingerprints", "mutag_188_target.txt")
print minmax(samples[0], samples[1])

0.475409836066


### 4. Algorithme du plus proche voisin

Créer une fonction ```find_nearest_neighbor``` qui calcule, étant donnés un dictionnaire ```dict_A```, une liste ```samples``` de dictionnaires, et une similarité (`tanimoto` ou `minmax`), l'indice du plus proche voisin de ```dict_A``` dans ```samples``` selon cette similarité.

In [6]:
def find_nearest_neighbor(dict_A, samples, similarity):
    """ Find the index of the nearest neighbor of dict_A in samples.
    
    Parameters:
      dict_A: dictionary 
        the keys are subgraph identifiers (indices in the original fingerprint file)
        and values are number of occurrences of the corresponding subgraph in object A.
      
      samples: list
        List containing as many elements as there are samples (i.e. molecules) in in_fingerprints_f.
        Each element is a dictionary in which the keys are subgraph identifiers (indices in the original fingerprint file)
        and the values the number of occurrences of the corresponding subgraph in the molecule.
      
      similarity: string
        'tanimoto' or 'minmax'
        
    Returns:
      index: int
         Index of the nearest neighbor of dict_A in samples.
    """
    if similarity == 'tanimoto':
        fn = tanimoto
    elif similarity == 'minmax':
        fn = minmax
    
    # Compute the list `similarities` of similarities of dict_A with each element of samples
    similarities = []
    for sample in samples:
        similarities.append(fn(dict_A, sample))
        
    # Return the index of the nearest neighbor of dict_A in samples,
    # i.e. of the max element in `similarities`
    
    return similarities.index(max(similarities))


Vérifier que le plus proche voisin de la première molécule de `mutag_188_CIRC_d2.fingerprints`, parmi toutes les *autres* molécules du même fichier, est celle qui a pour indice 32 (selon Tanimoto) ou 9 (selon MinMax)

In [7]:
#my_samples, my_labels = read_data("mutag_188_CIRC_d2.fingerprints", "mutag_188_target.txt")
print find_nearest_neighbor(samples[0], 
                            samples[1:], # we exclude '0' from the list
                            'tanimoto')
print find_nearest_neighbor(samples[0], 
                            samples[1:], # we exclude '0' from the list
                            'minmax')

32
9


Créer une fonction ```nearest_neighbor``` qui retourne l'étiquette du plus proche voisin d'un dictionnaire `dict_A` dans une liste de dictionnaires.

In [8]:
def nearest_neighbor(dict_A, samples, labels, similarity):
    """ Find the label of the nearest neighbor of dict_A in samples.
    
    Parameters:
      dict_A: dictionary 
        the keys are subgraph identifiers (indices in the original fingerprint file)
        and values are number of occurrences of the corresponding subgraph in object A.
      
      samples: list
        List containing as many elements as there are samples (i.e. molecules) in in_fingerprints_f.
        Each element is a dictionary in which the keys are subgraph identifiers (indices in the original fingerprint file)
        and the values the number of occurrences of the corresponding subgraph in the molecule.
      
      labels: list
        List containing as many elements as there are molecules in samples.
        Each element is the label of the corresponding molecule.
        The elements are ordered in the same way as in samples, 
        i.e. the label of samples[i] is given by labels[i]
        Each element is either +1 or -1.
      
      similarity: string
        'tanimoto' or 'minmax'
        
    Returns:
        predicted_label: signed int
         Label of the nearest neighbor of dict_A in samples.
    """
    index = find_nearest_neighbor(dict_A, samples, similarity)
    return labels[index]

### 5. Validation croisée

Créer une fonction `cross_validate_nearest_neignbor` qui 
1. coupe un jeu de données en k jeux d'entrainements et de validation selon le principe de la validation croisée 
1. pour chaque jeu de validation, prédit la classe d'une molécule en lui associant l'étiquette de son plus proche voisin *dans le jeu d'entrainement correspondant*
1. retourne la liste de ces prédictions, correctement ordonnées.

In [9]:
import random
def cross_validate_nearest_neighbor(num_folds, samples, labels, similarity):
    """ Cross-validate a nearest neighbor classifier on the given data.
    
    Parameters:
      num_folds: int
        Number of folds.
      samples: list
        List containing as many elements as there are samples (i.e. molecules) in in_fingerprints_f.
        Each element is a dictionary in which the keys are subgraph identifiers (indices in the original fingerprint file)
        and the values the number of occurrences of the corresponding subgraph in the molecule.
      
      labels: list
        List containing as many elements as there are molecules in samples.
        Each element is the label of the corresponding molecule.
        The elements are ordered in the same way as in samples, 
        i.e. the label of samples[i] is given by labels[i]
        Each element is either +1 or -1.
        
      similarity: string
        'tanimoto' or 'minmax'. 
        
    Returns:
      predictions: list
        List containing as many elements as there are molecules in samples.
        Each element is the predicted label of the corresponding molecule.
        The elements are ordered in the same way as in samples, 
        i.e. the label of samples[i] is given by labels[i]
        Each element is either +1 or -1.
    """
    # Split the dataset in k folds:
    # - Get the total number of samples
    n = len(samples)    
    # - Create the corresponding list of samples indices
    indices = range(n)
    # - Shuffle the list
    random.shuffle(indices)
    # - Split the list in k parts (this creates the lists of test indices)
    # take the complementary of the test indices as train indices
    folds = [] # folds[k] = {tr:list of train indices, te: list f test indices}
    fold_size = n/num_folds
    if fold_size*num_folds < n:
        # n is not a multiple of num_folds
        fold_size += 1    
    for k in range(num_folds):        
        this_fold = {} # {tr:list of train indices, te: list f test indices}
        this_fold['te'] = indices[k*fold_size:min((k+1)*fold_size, n)]
        this_fold['tr'] = list(set(indices) - set(this_fold['te']))
        this_fold['te'].sort()
        this_fold['tr'].sort()
        folds.append(this_fold)
    # print folds
    
    # Predict 
    predictions = [0]*n
    for k in range(num_folds):
        train_dict_list = [samples[i] for i in folds[k]['tr']]
        train_lab_list = [labels[i] for i in folds[k]['tr']]
        for test_idx in folds[k]['te']:
            test_dict = samples[test_idx]
            # predict the label of test_dict
            pred_lab = nearest_neighbor(test_dict,train_dict_list,train_lab_list,similarity)     
            # insert the predicted value at the right position in `predictions`
            predictions[test_idx]=pred_lab

    return predictions

In [10]:
#my_samples, my_labels = read_data("mutag_188_CIRC_d2.fingerprints", "mutag_188_target.txt")
my_predictions = cross_validate_nearest_neighbor(5, samples, labels, 'tanimoto')
my_predictions

['-1',
 '-1',
 '-1',
 '1',
 '-1',
 '1',
 '-1',
 '1',
 '-1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '-1',
 '1',
 '-1',
 '1',
 '-1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '-1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '-1',
 '-1',
 '1',
 '-1',
 '1',
 '1',
 '-1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '-1',
 '-1',
 '1',
 '1',
 '1',
 '1',
 '-1',
 '1',
 '1',
 '-1',
 '-1',
 '1',
 '-1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '-1',
 '1',
 '-1',
 '1',
 '-1',
 '-1',
 '1',
 '1',
 '1',
 '1',
 '-1',
 '1',
 '1',
 '1',
 '-1',
 '-1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '-1',
 '1',
 '1',
 '1',
 '1',
 '-1',
 '1',
 '1',
 '1',
 '1',
 '-1',
 '1',
 '1',
 '1',
 '1',
 '-1',
 '-1',
 '1',
 '-1',
 '-1',
 '1',
 '-1',
 '-1',
 '1',
 '-1',
 '-1',
 '1',
 '1',
 '-1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '-1',
 '-1',
 '-1',
 '-1',
 '-1',
 '1',
 '-1',
 '1',
 '1',
 '1',
 '-1',
 '1',
 '-1',
 '1',
 '-1',
 '-1',
 '-1',
 '-1',
 '-1',
 '1',
 '1',
 '-1',
 '-1',
 '1',
 '1',
 '-1',
 '-1',
 '-1',
 '1',
 

### 6. Évaluation des performances
Créer une fonction pour calculer chacune des mesures de performances suivantes : 
* accuracy
* precision
* recall

In [19]:
def accuracy(predicted_values, true_values):
    """Compute the accuracy of a prediction.
    
    Parameters:
      predicted_values: list
        List of predicted values. Each element is +1 or -1.
        
      true_values: list
        List of labels. Each element is +1 or -1.
        The elements are in the same order as in predicted_values.
        
    Returns:
      acc: float
        Accuracy of the prediction.
    """
    # Compute the number of true positives
    well_pred = [i  for i, true_val in enumerate(true_values) if true_val == predicted_values[i]]
    tp = len(well_pred)
    
    # Compute the total number of predictions
    total = len(true_values)
    
    # Compute the accuracy
    acc = float(tp)/float(total)

    return acc

In [24]:
def precision(predicted_values, true_values):
    """Compute the precision of a prediction.
    
    Parameters:
      predicted_values: list
        List of predicted values. Each element is +1 or -1.
        
      true_values: list
        List of labels. Each element is +1 or -1.
        The elements are in the same order as in predicted_values.
        
    Returns:
      prec: float
        Precision of the prediction.
    """
    # Compute the number of true positives
    well_pred = [i  for i, true_val in enumerate(true_values) if true_val == '1' and predicted_values[i] == '1']
    tp = len(well_pred)
    
    # Compute the number of predicted positives
    pred_pos = predicted_values.count('1')
    
    # Compute the accuracy
    prec = float(tp) / float(pred_pos)
    return prec


In [28]:
def recall(predicted_values, true_values):
    """Compute the recall of a prediction.
    
    Parameters:
      predicted_values: list
        List of predicted values. Each element is +1 or -1.
        
      true_values: list
        List of labels. Each element is +1 or -1.
        The elements are in the same order as in predicted_values.
        
    Returns:
      recall: float
        Recall of the prediction.
    """
    # Compute the number of true positives
    well_pred = [i for i, true_val in enumerate(true_values) if true_val == '1' and predicted_values[i] == '1']
    tp = len(well_pred)
    
    # Compute the number of false negatives
    fal_neg = [i for i, true_val in enumerate(true_values) if true_val == '1' and predicted_values[i] == '-1']
    fn = len(fal_neg)
    true_pos = true_values.count('1')
    
    # Compute the recall
    rec = float(tp)/(float(tp)+float(fn))
    return rec


Pour chacun des fichiers de fingerprints donnés, évaluer la performance de l'algorithme du plus proche voisin en termes de ces trois mesures.

In [30]:
filenames = ["mutag_188_CIRC_d2.fingerprints" ,
"mutag_188_PATH_d8.fingerprints",
"mutag_188_CIRC_d3.fingerprints" ,
"mutag_188_PATH_d4.fingerprints"]

for file in filenames:
    samples, labels = read_data(file, "mutag_188_target.txt")
    predictions = cross_validate_nearest_neighbor(5, samples, labels, 'minmax')
    print 'file: ', file
    print 'accuracy: ', accuracy(predictions, labels)
    print 'precision: ', precision(predictions, labels)
    print 'recall: ', recall(predictions, labels)

file:  mutag_188_CIRC_d2.fingerprints
accuracy:  0.88829787234
precision:  0.926229508197
recall:  0.904
file:  mutag_188_PATH_d8.fingerprints
accuracy:  0.872340425532
precision:  0.910569105691
recall:  0.896
file:  mutag_188_CIRC_d3.fingerprints
accuracy:  0.845744680851
precision:  0.875
recall:  0.896
file:  mutag_188_PATH_d4.fingerprints
accuracy:  0.867021276596
precision:  0.916666666667
recall:  0.88


### 7. Algorithme des k plus proches voisins
**Bonus** Remplacer l'algorithme du plus proche voisin par celui des k plus proches voisins. Faire varier k.